A short introduction to the challenge is given here: A gentle introduction to the world of DNA, mRNA and the mechanics of the Open Covid vaccine in plain English
https://www.kaggle.com/c/stanford-covid-vaccine/discussion/182320

The train data is a set of rows like:
{"index":0,"id":"id_001f94081","sequence":"GC","structure":"......","predicted_loop_type":"EEEE...","signal_to_noise":6.894,"SN_filter":1.0,
 "seq_length":107,"seq_scored":68,"reactivity_error":[0.1359,0.207...],"deg_error_Mg_pH10":[0.2613,0.0516...], "deg_error_pH10":[0.2631,0.1191..],
 "deg_error_Mg_50C":[0.1501,0.275...],"deg_error_50C":[0.2167,0.3475...], "reactivity":[0.3297,1.5693...],"deg_Mg_pH10":[0.7556,2.983...],
 "deg_pH10":[2.3375,3.506...],"deg_Mg_50C":[0.3581,0.4812...],"deg_50C":[0.6382,3.4773...]}

The test data is a set of rows like:
{"index":0,"id":"id_00073f8be","sequence":"AC","structure":"......","predicted_loop_type":"EEEE...","seq_length":107,"seq_scored":68}

We will investigate these columns in details later, but in general, each line contains a complete json object. Use lines=true when reading the Json files else we get a trailing error: https://stackoverflow.com/questions/61716765/pandas-read-json-trailing-data

We also install the forgi and the viennarna libraries which help to visualize the structure

In [ ]:
!conda install -y -c bioconda forgi;
!conda install -y -c bioconda viennarna;

import forgi.graph.bulge_graph as fgb 
import forgi.visual.mplotlib as fvm

import numpy as np
import pandas as pd

train = pd.read_json('../input/stanford-covid-vaccine/train.json', lines=True)
test = pd.read_json('../input/stanford-covid-vaccine/test.json', lines=True)

In [ ]:
print(' Train shape: ', train.shape, '\n', 'Test shape: ', test.shape)

2400 rows of training to 3634 of test !! Hosts want to avoid overfitting at all costs. Also possibly some of the sequences should not be in public domain. We have to rely on CV accuracy. It will play a big role here

In [ ]:
train.info()
train.head(1)

A short writeup on the i/p data is provided here:
Features in plain English - and some initial perspectives, random thoughts
https://www.kaggle.com/c/stanford-covid-vaccine/discussion/182839

In [ ]:
test.info()
test.head(1)

Nothing visibly wrong with data. No nulls as per quick overview. Let us look at specific data now
If you have read the links I had provided then you may be knowing that the real i/p is the sequence alone.
The redundant inputs are the structure and the predicted loop type.

In [ ]:
print('\n',train.iloc[0].sequence, '\n\n', train.iloc[0].structure, '\n\n', train.iloc[0].predicted_loop_type,)

For each base, we have the structure which tells us about the pairing of the base. The 'dot' means that base is not paired to anybody and is a weak link. Paired bases are indicated by '(' and ')'. The predicted loop type is the structure formed as a consequence of pairing. It can be a 'S'tem or an e'X'ternal loop etc. Please do refer to the earlier links for the details which are provided in simple English language.

Now let us take a look at the other columns. For EACH BASE in each row, we have a list containing the observed values of 'reactivity' and 'degradation'. These are the o/p variables that we must predict. For e.g. for row 1:

seq:GGAA...

reactivity: 0.3297, 1.5693000000000001, 1.1227, 0.8686...

deg_Mg_pH10: 0.7556, 2.983, 0.2526, 1.3789...

and so on. These reactivity and degradation cols are the labels and they are there for the first 68 bases. The rest of the bases are some sort of a marker and we wont be bothered about those.

It is interesting to note that the only real input to this ML problem is the Sequence field. The Structure and the PredictedLoopType are just predictions. These predictions can be calculated by us thru' external soures as well. Even the BPPM files given (we will come to that in a minute) are also predictions and can be calculated from external sources. So the only real input here is the Sequence. We can choose to discard the rest of the features or use them or search for better alternatives from external sources. The o/ps are the reactivity and the degradation values.

In general, we see that the better the reactivity (lower) the lower the degradation. Of course we have not considered errors so far. Errors in measuring reactivity and degradation rates may happen. We will investigate them later on.

Now let us try to pair up the bases. For every ')' that we encounter will map to the first '(' to the left of it. We will use queues. This may be the best way to get the pairs. I am sure the code can be improved

In [ ]:
train['seqpair']=[train.iloc[i]['structure'].replace('.', 'X') for i in range(len(train))]
train['seqpairpos']=None
train['seqpairpos_dist']=None

def Sort_Tuple(tup):  
    ##i/p: list of tuples. o/p: sorted list
    tup.sort(key = lambda x: x[0])  
    return tup  
    ##strange why return tup.sort(key = lambda x: x[0]) does not work

def pairup(row):
    ##take each row and pair the bases, create positions maps also
    queue = []
    seqpair=list(row.seqpair)
    seqpairposmap = []
    seqpairpos_dist = []

    for i in range(len(row.structure)) :
        if row.structure[i]=='(':
            queue.append(i) 
        if row.structure[i]==')':
            pairpos = queue.pop()
            ##i'th base is paired with pairpos'th base
            seqpair[pairpos] = row.sequence[i]
            seqpair[i] = row.sequence[pairpos]
            seqpairposmap.append((i,pairpos))
            seqpairposmap.append((pairpos,i))
        if row.structure[i]=='.':
            ##unpaired bases. They map to themselves
            seqpairposmap.append((i,i))
 
    train.loc[train.id==row.id, 'seqpairpos'] = [Sort_Tuple(seqpairposmap)]
    ##If you have strong masachotic tendencies :) try - Sort_Tuple(seqpairpos) without [] braces
    ##for some reason the loop repeats twice for every record
    
    ##Also find distance between pairs
    explodedpair=train[train.id==row.id]['seqpairpos'].explode().tolist()
    [seqpairpos_dist.append(abs(explodedpair[counter][1]-explodedpair[counter][0])) for counter in range (len(explodedpair))]

    train.loc[train.id==row.id, 'seqpair'] = ''.join(seqpair)
    ##An ugly hack. Let me know if there is a better way. Pandas just does not want to store lists in cells
    train.loc[train.id==row.id, 'seqpairpos_dist'] = pd.Series([seqpairpos_dist] * len(train))

train.apply(pairup, axis=1)
print('\n',train.iloc[0].sequence, '\n', train.iloc[0].structure, '\n', train.iloc[0].predicted_loop_type, '\n', train.iloc[0].seqpair, '\n\nDistance\n', train.iloc[0].seqpairpos_dist, '\n\nPairPos\n', train.iloc[0].seqpairpos)

Much better. We see who is paired to whom. C to G and A to U usually but apparently there are also wobble pairs. So it is better to create this pairing. We can always discard it if it does not improve the scores

Also now we have the pairing distance. Some pairings are near and some are far. The ones that are far means that the RNA has folded back completely on itself..maybe it indicates whether the fold is deep or not. We are not sure if this info is important but it can be fed as an additional i/p to the model. The 'distance' indicates in some way how the RNA has 'folded' and 'maybe' this can help with predicting the o/p variable. We may want to categorize the distance as very near, near, far, v.far or try some other categorization.

Now let us turn our attension to the BPPM folder. These are also redundant files. Every row has a file. Each file contains a N into N matrix info on how much probability a base in that row has to pair up with every other base in that row. Looking at the file will make things clearer. 

In [ ]:
import matplotlib.pyplot as plt
from pathlib import Path
   
bppm=np.load(Path("../input/stanford-covid-vaccine/bpps") / f"{train.loc[0].id}.npy")
plt.figure(figsize = (12, 8))
plt.imshow(bppm)
plt.colorbar()

This is Color coding for probability of pairing between each base with every other base for the first mRNA sequence. Now let us see what the actual structure is and try to plot it.

In [ ]:
##Set all zeros
predicted = np.zeros((len(train.iloc[0].structure), len(train.iloc[0].structure)))
pairs = []

##Set BPPM=1 for all paired bases as given in Structure column
for i, structure in enumerate(train.iloc[0].structure):
    if structure == "(":
        pairs.append(i)
    elif structure == ")":
        j = pairs.pop()
        predicted[i, j] = 1
        predicted[j, i] = 1

f, axarr = plt.subplots(1,2,figsize = (20, 10))
axarr[0].imshow(bppm)
axarr[1].imshow(predicted)

Definitely looks like there are additional structural possibilities also. For e.g. see the graph on the left the 2nd diaganol line. That is a strong possibility

There are couple options, I can think of: 
1. Sum up the probabilities of pairing for each base and use this i/p as addition to the model
2. Get alternate structures and from that calculate 'predictedlooptype'. Feed 2-3 alternate strcutures as i/p to the model along with the primary one

Another point. We can have reverse segments (mirror pairs), typically stems. These ideally should have the same reactivity. We can tokenize these.

But first let us take a look at how the mRNA molecule is formed. We need to download the forgi library

In [ ]:
plt.figure(figsize = (18, 18))
fvm.plot_rna(fgb.BulgeGraph.from_fasta_text(f'>rna1\n{train.iloc[0].structure}\n{train.iloc[0].sequence}')[0])
plt.show()

pd.set_option('display.max_colwidth', -1)
print('\n',train.iloc[0].sequence, '\n', train.iloc[0].structure, '\n', train.iloc[0].predicted_loop_type, '\n\n', \
      train.iloc[0].seqpair, '\n\nDistance\n', train.iloc[0].seqpairpos_dist, '\n\nPairPos\n', train.iloc[0].seqpairpos)
pd.reset_option('display.max_colwidth')

This is a 2D image. Every 10 bases there is a 360 deg swift in the molecule. There could also be pseudo knots which are not effectively captured here. And of course this is a simplisting representation. Each base itself is a molecule which chemically interacts with the other bases in different ways

The diff shapes formed (predictedlooptype) are highlighted in different colors. The stems are the paired ones and apparently the most stable parts, less prone to degradation. We can see the reverse sequences in the molecule formed in stems (green). I believe it makes sense to capture this information as a separate feature. During modelling, it makes sense to look at the reverse complement when focussing on any base sequence.

So let us find the reverse complement tokens for the above structure. We can visually see 5 of them but actually looking at the structure there are 6. The 5th and 6th base pairings (with 5-24,6-23) are not continuous with the 7,8,9,10. The 22nd base breaks the sequence. So it can be broken into 2 reverse complements - 5,6 and 7,8,9,10.

.......((((.......)))).((((((........))))))((..((((..........)))))..(((((((....))))))).....................
GGAAAAGCUCUAAUAACAGGAGACUAGGACUACGUAUUUCUAGGUAACUGGAAUAACCCAUACCAGCAGUUAGAGUUCGCUCUAACAAAAGAAACAACAACAACAAC


- Find the instances of ')'. Let this be 'i'
- walk forward till character changes to something else. Let this pos be 'j'
- walk backwards till the first '(' encountered. assume this is 'b'
- keep walking backwards till character changes. This position is 'a'


- Take min of 'b-a' and 'j-i'. Let this be 'x'
- The substring from 'b' to 'b-x' is the segment
- The substring from 'i' to 'i+x' is the r-seg

Maybe there is a more 'Pythonic' way. If so please excuse the style of coding

In [ ]:
rc_dict=[]
train['rc']=train['sequence']

def findRC(row):
    queue = []
    restartfrom=0
    token=''
    tok=0
    rc=row.rc
                                    
    for i in range(len(row.structure)) :
        if restartfrom>i:
            continue
    
        if row.structure[i]=='(':
            queue.append(i) 
    
        if row.structure[i]==')':
            for j in range(i,len(row.structure)):
                if row.structure[j] != ')':
                    break;

            length=j-i
            for a in range(j-i):
                b = queue.pop()
            if length>1:
                ##print('seg start:', b+1, ' seg length:', length, 'seg:',row.sequence[b:b+length])
                ##print('rseg start:', i+1, ' rseg length:', length, 'rseg:',row.sequence[i:i+length], '\n')
                rc=rc[0:b]+''.join([token+str(tok) for x in range(length)])+rc[b+length:i]+''.join([token+str(tok) for x in range(length)])+rc[i+length:]
                tok=tok+1
                restartfrom=i+length
                rc_dict.append(row.sequence[b:b+length])

    ##print(rc)
    print('More than 10 tokens') if (tok-1)>9 else None
    train.loc[train.id==row.id, 'rc'] = rc
    
train.apply(findRC, axis=1)
print('\n',train.iloc[0].sequence, '\n', train.iloc[0].structure, '\n', train.iloc[0].predicted_loop_type, '\n', train.iloc[0].seqpair, '\n', train.iloc[0].rc)

There could be 2 aspects here
- We could have separate word embeddings for these sequence of RCs across the entire datset
- Alternatively we can try to bring in some attention mechanism when processing the RCs for each row so that model is able to treat these specially

Let us now also calculate the probability of pairing for each base. This is found by simply adding all BPPMs for that base. Let us track the actual pairing also as 0's and 1s. The actual pairing can be found from the structure col

In [ ]:
def findprob(row):
    bppm=np.load(Path("../input/stanford-covid-vaccine/bpps") / f"{row.id}.npy")
    train.loc[train.id==row.id, 'pair_prob'] = pd.Series([np.sum(bppm, axis=0).tolist()] * len(train))
    ##the ugly hack again above. Somehow unable to get list into a cell without this hack

    pair=[]
    for i, structure in enumerate(row.structure):
        pair.append('1') if structure!='.' else pair.append('0')
    train.loc[train.id==row.id, 'paired'] = ''.join(pair)
    
train.apply(findprob, axis=1);

Let us now try to find out if the features we have introduced make sense:
- Do base-pairs have similar reactivity / degradation values. Ideally they should have.
- Probability of pairing versus actual pairing
- We can even try to compare reactivity / degradation values for RC pairs across all rows. Ideally they should somewhat be similiar (I hope?)

In [ ]:
pd.set_option('display.max_colwidth', -1)
print('\n',train.iloc[0].sequence[:25], '\n', train.iloc[0].structure[:25], '\n', train.iloc[0].predicted_loop_type[:25], '\n\n', \
      train.iloc[0].seqpair[:25], '\n', train.iloc[0].rc[:25], '\n', train.iloc[0].paired[:25], '\n\n', train.iloc[0].pair_prob[:25])
pd.reset_option('display.max_colwidth')

plt.figure(figsize=(15,5))
plt.plot(train.iloc[0]['reactivity'])
plt.plot(train.iloc[0]['deg_Mg_50C'])

- Do base-pairs have similar reactivity / degradation values. Ideally they should have.
- Probability of pairing versus actual pairing
- We can even try to compare reactivity / degradation values for RC pairs across all rows. Ideally they should somewhat be similiar (I hope?)

The first question is answered a little down the line

The probability of pairing loosely matches with the actual pairing, but we can clearly see that alternate structures are very much possible (with slightly higher energy). So we need to generate at least a couple of alternate structures for each row. We do that later

In general we see that reactivity and degradation follow a similar pattern. Higher the reactivity, more unstable the pair is and this results in higher degradation

It would be nice to expand the inner lists to do further EDA. There is a neat bit of code by @HidehisaArai to explode the lists. I just added couple of lines to it to split the sequence as well and retain a separate col 
as permanent index for easier queries during EDA.

In [ ]:
##convert sequences to list so it can explode
train['seqsplit']=[list(train.iloc[i]['sequence'][:68]) for i in range(len(train))]
train['seqpair']=[list(train.iloc[i]['seqpair'][:68]) for i in range(len(train))]
train['rc']=[list(train.iloc[i]['rc'][:68]) for i in range(len(train))]
train['paired']=[list(train.iloc[i]['paired'][:68]) for i in range(len(train))]
train['pair_prob']=[list(train.iloc[i]['pair_prob'][:68]) for i in range(len(train))]
train['seqpairpos']=[list(train.iloc[i]['seqpairpos'][:68]) for i in range(len(train))]
train['seqpairpos_dist']=[list(train.iloc[i]['seqpairpos_dist'][:68]) for i in range(len(train))]
##add an index col
train['numbering']=[np.arange(68).tolist() for i in range(len(train))]

##select all cols to be exploded
list_cols = ['seqsplit','numbering','seqpair','rc', 'paired','pair_prob','seqpairpos', 'seqpairpos_dist']
list_cols += train.filter(like="reactivity", axis=1).columns.tolist()
list_cols += train.filter(like="deg", axis=1).columns.tolist()
other_cols = list(set(train.columns) - set(list_cols))

exploded = [train[col].explode() for col in list_cols]
train_expanded = pd.DataFrame(dict(zip(list_cols, exploded)))
train_expanded = train[other_cols].merge(train_expanded, how="right", left_index=True, right_index=True).reset_index(drop=True)

In [ ]:
int_cols = ['numbering','seqsplit','seqpair', 'seqpairpos', 'seqpairpos_dist','paired','pair_prob','reactivity','deg_Mg_pH10','deg_Mg_50C']
df=train_expanded[train_expanded.id=='id_001f94081'][int_cols]
display(df.transpose())

Let us now check if paired bases have better reactivity and degradation values

In [ ]:
print('Mean Reactivity of paired base', round(df[df.seqpair!='X']['reactivity'].mean(),3))
print('Mean Reactivity of unpaired base', round(df[df.seqpair=='X']['reactivity'].mean(),3))
print('\nMean degradation(temp) of paired base', round(df[df.seqpair!='X']['deg_Mg_50C'].mean(),3))
print('Mean degradation(temp) of unpaired base', round(df[df.seqpair=='X']['deg_Mg_50C'].mean(),3))
print('\nMean degradation(Ph) of paired base', round(df[df.seqpair!='X']['deg_Mg_pH10'].mean(),3))
print('Mean degradation(Ph) of unpaired base', round(df[df.seqpair=='X']['deg_Mg_pH10'].mean(),3))

The reactivity of unpaired bases is twice that of paired ones. So they are unstable? and this accounts for their faster(higher) degradation??

How about paired bases? If say 'C is paired to G', then are their degradation and reactivity similar to each other? Ideally they should be

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df[df.seqpair!='X'].transpose())

Hmm. Not really. One of our assumption was wrong. So even if a base pair is formed, the 2 partners could have varying reactivity and degradation...though as we saw on the whole the reactivity and degradation improve.

In [ ]:
pd.reset_option('all')

from collections import Counter
rc_list=Counter((rc_dict)).most_common()
list(map(lambda x: print(x) if (len(x[0])>4) & (x[1]>8) else None, rc_list));

There are a LOT of repeated patterns!! Can we create custom word embeddings for these patterns? Would that give a better score rather than processing character by character?

Coming to alternate secondary structures, I spend a lot of time conjuring up code. Finally I got something that will generate the structure about half the time. But looks like Prof Rhiju's Github already has the necessary code to generate alternate structures:
https://github.com/DasLab/arnie/blob/master/notebooks/start_here.ipynb

Predicted looptypes for these alternate structures can be found here:
https://github.com/hendrixlab/bpRNA

Lastly let us look at the labels across the entire training data instead of looking at just 1 row and see if we can find something unique. Let us take the means

In [ ]:
avg_reactivity = np.array(list(map(np.array,train.reactivity))).mean(axis=0)
avg_deg_Mg_50C = np.array(list(map(np.array,train.deg_Mg_50C))).mean(axis=0)
avg_deg_Mg_pH10 = np.array(list(map(np.array,train.deg_Mg_pH10))).mean(axis=0)

plt.figure(figsize=(18,6))
plt.plot(avg_reactivity)
plt.plot(avg_deg_Mg_50C)
plt.plot(avg_deg_Mg_pH10)

plt.show()

In [ ]:
avg_deg_error_Mg_pH10 = np.array(list(map(np.array,train.deg_error_Mg_pH10))).mean(axis=0)
avg_deg_error_Mg_50C = np.array(list(map(np.array,train.deg_error_Mg_50C))).mean(axis=0)
avg_reactivity_error = np.array(list(map(np.array,train.reactivity_error))).mean(axis=0)

plt.figure(figsize=(18,6))
plt.plot(avg_reactivity_error)
plt.plot(avg_deg_error_Mg_50C)
plt.plot(avg_deg_error_Mg_pH10)

plt.show()

The first 5 bases are highly error prone as well are extremely degradable! The loops and structure would not even have started at that point. Possibly this explains the high levels of instability. But measuring them seems to be error prone as can be seen from the error graphs. One option could be to club the first 5 bases as one word and create embeddings accordingly. Or probably use a separate model altogether for these 5 bases? The rest seems relatively easily predictable

This is just an exploratory EDA and the new features may/may not help. The test data size is vast here compared to the traindata. Overfitting is a major concern. If you are generating new features, please exercise caution. Though they could improve the CV/public scores, we dont know the effect on the private data set. For e.g. taking the bppm sum to generate P(pairing) seems to be a good idea but the matrix pairs/distance are not distributed in similar pattern in the train/pub-test and potentially private test also. So exercise caution while using those and they may need some normalizing like dividing by the length or something even better to ensure similar distribution pattern..Thanks @its7171 for pointing that out.. It is an important disclaimer to keep in mind

Onto the model now...